Welcome

In [11]:
#dependencies needed for s3.py script
#RUN EVERYTIME YOU CONNECT TO NEW RUN TIME

!pip install boto3
!pip install python-dotenv
!pip install transformers
!pip install torch

In [12]:
#imports needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from dotenv import load_dotenv
import sys

#roberta
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification


In [13]:
#connect to google drive to access script and .env file for s3
#for this to work - you need to add a shortcut to the group project folder to 'MyDrive' (can do this by clicking organise)

#mount google drive to access required files
from google.colab import drive
drive.mount('/content/drive')

#.env file needed for s3 script
env_path = "content/drive/MyDrive/Group_project/Code/.env"
load_dotenv(env_path)

# #s3 script
%cd /content/drive/MyDrive/Group_project/Code/
import s3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1ouUVa9cO_b84XmO7kdkhqpgoEh4mM3EI/Group project/Code


Change the path to data below as required

In [14]:
s3.download("processed/news/gnews/Accenture.parquet")
df = pd.read_parquet(s3.s3_to_local_path("processed/news/gnews/Accenture.parquet"))

In [15]:
df['full_data']= df['title']+ ' ' + df['body'].astype(str)
# title_data = df[['title', 'dt']]
# body_data = df[['body', 'dt']]

In [17]:
#RoBERTa model

#connect to GPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using GPU:", torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print("Using CPU")

#model name
#MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
MODEL = f"ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

# Function to get sentiment scores for a chunk
def get_sentiment_scores(chunk):
    encoded_text = tokenizer(chunk, return_tensors='pt')
    output = model(**encoded_text)
    #extracting and normalising sentiment scores
    scores = output[0][0].detach().numpy()
    #normalising the score
    return softmax(scores)

#function to split chunks if needed since roberta can do 512 at max (max 512 tokens can be processed at a time)
def analyse_large_text(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)  # Encode to token IDs without special tokens
    chunk_size = 512
    # Overlap between chunks to avoid missing context between chunks
    stride = 256
    sentiment_scores = []
    token_lengths = []

    for i in range(0, len(tokens), stride):
        chunk = tokens[i:min(i + chunk_size, len(tokens))]
        chunk_text = tokenizer.decode(chunk)  # Decode back to text

        inputs = tokenizer(chunk_text, return_tensors='pt', truncation=True, max_length=512).to(device)
        output = model(**inputs)
        scores = output[0][0].detach().cpu().numpy()
        sentiment_scores.append(scores)
        token_lengths.append(len(chunk))

    # Weighted average of scores by chunk length
    sentiment_scores = np.array(sentiment_scores)
    weighted_scores = np.average(sentiment_scores, axis=0, weights=token_lengths)
    # print('sentiment scores \n', sentiment_scores)
    # print('weighted scores \n', weighted_scores)

    # Return final aggregated sentiment
    return {
        'roberta_pos': softmax(weighted_scores)[2],
        'roberta_neu': softmax(weighted_scores)[1],
        'roberta_neg': softmax(weighted_scores)[0],
    }

df[['roberta_pos', 'roberta_neu', 'roberta_neg']] = df['full_data'].apply(lambda x: pd.Series(analyse_large_text(x)))
# title_data[['roberta_pos', 'roberta_neu', 'roberta_neg']] = title_data['title'].apply(lambda x: pd.Series(analyse_large_text(x)))
# body_data[['roberta_pos', 'roberta_neu', 'roberta_neg']] = body_data['body'].apply(lambda x: pd.Series(analyse_large_text(x)))

Using GPU: Tesla T4


Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
#Roberta ENSEMBLE MODEL

# # Device setup (GPU if available, otherwise CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load both sentiment models
# MODEL_1 = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # Social media sentiment model
# MODEL_2 = "ProsusAI/finbert"  # Financial sentiment model

# tokenizer_1 = AutoTokenizer.from_pretrained(MODEL_1)
# model_1 = AutoModelForSequenceClassification.from_pretrained(MODEL_1).to(device)

# tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_2)
# model_2 = AutoModelForSequenceClassification.from_pretrained(MODEL_2).to(device)

# # Function to get sentiment scores for a given model
# def get_sentiment_scores(model, tokenizer, text):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
#     output = model(**inputs)
#     scores = output.logits.detach().cpu().numpy()[0]
#     return softmax(scores)  # Normalize the scores

# # Function to analyse large text chunks
# def analyse_large_text(text):
#     tokens = tokenizer_1.encode(text, add_special_tokens=False)
#     chunk_size = 512
#     stride = 256
#     sentiment_scores_1 = []
#     sentiment_scores_2 = []
#     token_lengths = []

#     for i in range(0, len(tokens), stride):
#         chunk = tokens[i:min(i + chunk_size, len(tokens))]
#         chunk_text = tokenizer_1.decode(chunk)

#         # Get sentiment scores from both models
#         scores_1 = get_sentiment_scores(model_1, tokenizer_1, chunk_text)
#         scores_2 = get_sentiment_scores(model_2, tokenizer_2, chunk_text)

#         sentiment_scores_1.append(scores_1)
#         sentiment_scores_2.append(scores_2)
#         token_lengths.append(len(chunk))

#     # Weighted average of scores (based on token lengths)
#     sentiment_scores_1 = np.array(sentiment_scores_1)
#     sentiment_scores_2 = np.array(sentiment_scores_2)

#     weighted_scores_1 = np.average(sentiment_scores_1, axis=0, weights=token_lengths)
#     weighted_scores_2 = np.average(sentiment_scores_2, axis=0, weights=token_lengths)

#     # Final ensemble output (Average scores from both models)
#     final_scores = (weighted_scores_1 + weighted_scores_2) / 2

#     # Return final sentiment scores
#     return {
#         'ensemble_pos': final_scores[2],  # Positive
#         'ensemble_neu': final_scores[1],  # Neutral
#         'ensemble_neg': final_scores[0],  # Negative
#     }

# # Apply sentiment analysis to title and body data
# title_data[['ensemble_pos', 'ensemble_neu', 'ensemble_neg']] = title_data['title'].apply(lambda x: pd.Series(analyse_large_text(x)))
# body_data[['ensemble_pos', 'ensemble_neu', 'ensemble_neg']] = body_data['body'].apply(lambda x: pd.Series(analyse_large_text(x)))


In [ ]:
# #save as csv files
# from google.colab import files

# #change file names as required
# title_data.to_csv('GS_title_model3.csv', encoding='utf-8')
# body_data.to_csv('GS_body_model3.csv', encoding='utf-8')

# files.download('GS_title_model3.csv')
# files.download('GS_body_model3.csv')

In [18]:
display(df)
# display(title_data)
# display(body_data)

,title,description,body,url,domain,dt,full_data,roberta_pos,roberta_neu,roberta_neg
0,ACN Stock: Accenture Tops Targets But Lowers E...,Accenture beat Wall Street's targets for its f...,Accenture (ACN) on Thursday beat Wall Street's...,https://www.investors.com/news/technology/acn-...,Investor's Business Daily,1734611450000,ACN Stock: Accenture Tops Targets But Lowers E...,0.087462,0.034915,0.877622
1,Accenture plc (ACN) reports earnings,"The report was filed on December 19, 2024",In This Story ACN -2.27%\nAccenture plc (ACN-2...,https://qz.com/accenture-plc-acn-reports-earni...,Quartz India,1734609821000,Accenture plc (ACN) reports earnings In This S...,0.076803,0.172987,0.750211
2,A bearish bet on a stock whose chart is raisin...,Nishant Pant breaks down a trade in consulting...,"Two weeks ago, I had covered a trade on Alibab...",https://www.cnbc.com/2024/10/29/a-bearish-bet-...,CNBC,1730215955000,A bearish bet on a stock whose chart is raisin...,0.912116,0.055175,0.032709
3,"5 steps firms should take to see ROI from AI, ...","As companies expand the use of generative AI, ...",Good morning. As CFOs determine the business c...,https://fortune.com/2024/10/14/5-steps-firms-c...,Fortune,1728878400000,"5 steps firms should take to see ROI from AI, ...",0.515184,0.353036,0.131780
4,Accenture bets big on generative AI with Nvidi...,Accenture and Nvidia have announced they are e...,A hot potato: Artificial intelligence is a hot...,https://www.techspot.com/news/104987-accenture...,TechSpot,1727988600000,Accenture bets big on generative AI with Nvidi...,0.654501,0.009377,0.336122
...,...,...,...,...,...,...,...,...,...,...
195,"In Times Of Increasing Volatility, 2 ETFs To P...",Market Analysis by Investing.com (Tezcan Gecgi...,It's been a rough few days for stock market in...,https://www.investing.com/analysis/in-times-of...,Investing.com,1620705600000,"In Times Of Increasing Volatility, 2 ETFs To P...",0.807396,0.021374,0.171230
196,"JOIN US JUNE 24: Execs from Deutsche Bank, All...","Melissa Fridman (left), Kate Burke (center), M...","Melissa Fridman (left), Kate Burke (center), M...",https://markets.businessinsider.com/news/stock...,Business Insider,1620619200000,"JOIN US JUNE 24: Execs from Deutsche Bank, All...",0.929400,0.029431,0.041169
197,The gaming industry’s 'duty of care' in keepin...,"Leaders from Roblox, Fair Play Alliance, and A...","On the last day of GamesBeat Summit last week,...",https://venturebeat.com/2021/05/05/the-gaming-...,VentureBeat,1620225750000,The gaming industry’s 'duty of care' in keepin...,0.866344,0.034123,0.099532
198,Accenture To Buy French Cybersecurity Services...,"(RTTNews) - Accenture (ACN), a professional se...","(RTTNews) - Accenture (ACN), a professional se...",https://markets.businessinsider.com/news/stock...,Business Insider,1619685015000,Accenture To Buy French Cybersecurity Services...,0.450949,0.009486,0.539565


In [19]:
df['dt'] = pd.to_datetime(df['dt'], unit='ms')
df['dt'] = df['dt'].dt.date

